In [3]:
from dnn_structure import *

import numpy as np
import sys
sys.path.append("../src")
from Load_data.xml_utils import parse_cvat_xml_all_labels, label_Y_binary
from Load_data.data_loader import load_and_resize_images, build_label_array


## prepare Data

In [4]:

TRAIN_XML = "../EIDSeg_Dataset/data/train/train.xml"
TEST_XML = "../EIDSeg_Dataset/data/test/test.xml"
TRAIN_IMAGES = "../EIDSeg_Dataset/data/train/images/default"
TEST_IMAGES = "../EIDSeg_Dataset/data/test/images/default"

IMAGE_SIZE = (64, 64)
NUM_ITER = 2000



labels_train_raw = parse_cvat_xml_all_labels(TRAIN_XML)
labels_test_raw = parse_cvat_xml_all_labels(TEST_XML)

Y_train_map = label_Y_binary(labels_train_raw)
Y_test_map = label_Y_binary(labels_test_raw)

X_train_org, ordered_train = load_and_resize_images(TRAIN_IMAGES, size=IMAGE_SIZE)
X_test_org, ordered_test = load_and_resize_images(TEST_IMAGES, size=IMAGE_SIZE)

Y_train_org = build_label_array(ordered_train, Y_train_map)
Y_test_org = build_label_array(ordered_test, Y_test_map)

train_x = X_train_org.reshape(X_train_org.shape[0], -1).T
test_x = X_test_org.reshape(X_test_org.shape[0], -1).T

print(train_x.shape, Y_train_org.shape)
print(test_x.shape, Y_test_org.shape)

Final X shape: (2612, 64, 64, 3)
Final X shape: (327, 64, 64, 3)
(12288, 2612) (1, 2612)
(12288, 327) (1, 327)


## try with He Initialization


In [5]:
def initialize_parameters_he(layers_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layers_dims) - 1 # integer representing the number of layers
     
    for l in range(1, L + 1):
        #(≈ 2 lines of code)
        # parameters['W' + str(l)] = 
        # parameters['b' + str(l)] =
        # YOUR CODE STARTS HERE
        parameters['W' + str(l)] = np.random.randn(layers_dims[l],layers_dims[l-1])*np.sqrt(2./layers_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layers_dims[l],1))
        # YOUR CODE ENDS HERE
        
    return parameters

## L-layer Neural Network


In [6]:
layers_dims = [12288, 20, 7, 5, 1] 

In [7]:

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (n_x, number of examples)
    Y -- true "label" vector (containing 1 if cat, 0 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization.

    #parameters = initialize_parameters_deep(layers_dims)
    parameters = initialize_parameters_he(layers_dims)    

    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)
                
        # Compute cost.
        cost = compute_cost(AL,Y)
            
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)
        
        # Update parameters.

        parameters = update_parameters(parameters, grads, learning_rate)
                        
        # Print the cost every 100 iterations and for the last iteration
        if print_cost and (i % 100 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)
    
    return parameters, costs

In [8]:
parameters, costs = L_layer_model(train_x, Y_train_org, layers_dims,learning_rate = 0.008, num_iterations = 3500, print_cost = True)

Cost after iteration 0: 0.630831367322515
Cost after iteration 100: 0.5836319288235235
Cost after iteration 200: 0.5705133220761593
Cost after iteration 300: 0.5598484707624054
Cost after iteration 400: 0.5497441521575789
Cost after iteration 500: 0.5403865494585717
Cost after iteration 600: 0.5324793933328509
Cost after iteration 700: 0.5222834897606259
Cost after iteration 800: 0.5119323279136482
Cost after iteration 900: 0.5023954375445786
Cost after iteration 1000: 0.49448971970050604
Cost after iteration 1100: 0.48762319360672135
Cost after iteration 1200: 0.4754437438932869
Cost after iteration 1300: 0.46351046337601165
Cost after iteration 1400: 0.46139507788646106
Cost after iteration 1500: 0.4440273817174804
Cost after iteration 1600: 0.44602558245324214
Cost after iteration 1700: 0.43780766058158754
Cost after iteration 1800: 0.42075597287920696
Cost after iteration 1900: 0.4366925115784739
Cost after iteration 2000: 0.39328563719708615
Cost after iteration 2100: 0.3766727229

In [9]:
pred_train = predict(train_x, Y_train_org, parameters)

Accuracy: 0.7901990811638593


In [10]:
pred_test = predict(test_x, Y_test_org, parameters)

Accuracy: 0.7370030581039756
